Stock Analysis Recommender. This solution can be extended with a real-time API for multiple stock symbols. In this example, analysis for only 1 stock IBM is displayed.

Disclaimer: This is just a test project. Please dont use it for any commercial use.

In [2]:
import os

import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr


In [3]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

In [4]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-7-sonnet-20250219"

In [36]:
system_message = """You are an assistant that recommends whether to buy or sell a stock that user provides. Give your best educated guess on the basis of the stock
 details provided to you like stock price, open, close, high, low price and technical indicators like SMA, EMA and RSI. Don't force an answer, only give the answer if the data you 
 receive is valid. If not valid or if you receive empty data, tell the user that the data received was not valid politely. Respond only in Markdown """

In [46]:
def user_prompt_for(data):
    user_prompt = f""" Recommend whether to buy or sell stock of IBM tomorrow. Here is the past 100 days of stock prices data which includes the stock
    open, close, high and low price for the day.\n\n {data['stock_data']} \n\n. 
    And here is the technical indicators data for the stock for past 52 weeks.\n\n
    Indicator Name: SMA
    Indicator Data: {data['SMA']} \n\n
    Indicator Name: EMA
    Indicator Data: {data['EMA']} \n\n
    Indicator Name: RSI
    Indicator Data: {data['RSI']} \n\n
    Analyze all this information and give your best educated guess based on the above data provided.
     """
    
    return user_prompt

In [49]:
def fetchData(stock_name):
    data = {}
    # Hardcoding Stock Data as the API costs as massive.
    data['stock_name'] = stock_name

    data['stock_data'] = stock_data["Time Series (Daily)"]

    technical_sma_data = sma_data['Technical Analysis: SMA']
    technical_ema_data = ema_data['Technical Analysis: EMA']

    technical_rsi_data = rsi_data['Technical Analysis: RSI']
    
    data['SMA'] = technical_sma_data
    data['EMA'] = technical_ema_data
    data['RSI'] = technical_rsi_data

    return data




In [38]:
def messages_for(stock):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(fetchData(stock))}
    ]

In [47]:
def stream_gpt(stock):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(stock), stream=True)
    reply = ""
    
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)

In [50]:
display_handle = display(Markdown(""), display_id=True)
stream_gpt('IBM')

Based on the data provided for IBM's stock, here are a few considerations:

1. **Price Movement:**
   - The recent price is $288.37, with a gradual upward trend from $279.80 (5 days ago) to $288.37.

2. **SMA and EMA Analysis:**
   - The 52-week Simple Moving Average (SMA) on the latest date is $252.8374.
   - The Exponential Moving Average (EMA) for the same period is $260.7207.
   - Both SMA and EMA are below the current stock price, which can suggest a bullish trend as the price is above these moving averages.

3. **RSI:**
   - The Relative Strength Index (RSI) as of the latest date is 67.6313.
   - This suggests that the stock is nearing overbought conditions (traditionally above 70 is considered overbought), but not quite there yet.

**Recommendation:**
- The current upward trend, along with the price being above SMA and EMA, and the RSI approaching overbought levels, suggests a cautious optimism about further price increases.
- If you're holding IBM stock, you might consider continuing to hold, but be wary of the RSI indicating a potential for near-term overbought conditions. 
- For new entries, it might be wise to wait for a potential dip before buying in, or ensure tight stop-losses are in place if buying now to protect against sudden reversals.

Please assess these insights with additional fundamental research and individual financial goals before making investment decisions.

In [42]:
stock_data = {"Meta Data":{"1. Information":"Daily Prices (open, high, low, close) and Volumes","2. Symbol":"IBM","3. Last Refreshed":"2025-10-03","4. Output Size":"Compact","5. Time Zone":"US/Eastern"},"Time Series (Daily)":{"2025-10-03":{"1. open":"287.5000","2. high":"293.3200","3. low":"287.3000","4. close":"288.3700","5. volume":"4375082"},"2025-10-02":{"1. open":"285.7900","2. high":"288.5400","3. low":"282.7900","4. close":"286.7200","5. volume":"3814232"},"2025-10-01":{"1. open":"280.2000","2. high":"286.5900","3. low":"280.1500","4. close":"286.4900","5. volume":"4381338"},"2025-09-30":{"1. open":"280.8800","2. high":"286.0250","3. low":"280.5200","4. close":"282.1600","5. volume":"5926924"},"2025-09-29":{"1. open":"286.0000","2. high":"286.0000","3. low":"279.6600","4. close":"279.8000","5. volume":"6022125"},"2025-09-26":{"1. open":"280.5100","2. high":"288.8500","3. low":"280.1100","4. close":"284.3100","5. volume":"9063938"},"2025-09-25":{"1. open":"272.9350","2. high":"284.2300","3. low":"271.1480","4. close":"281.4400","5. volume":"11506192"},"2025-09-24":{"1. open":"272.6200","2. high":"273.6499","3. low":"267.3000","4. close":"267.5300","5. volume":"3159924"},"2025-09-23":{"1. open":"272.7000","2. high":"273.2962","3. low":"269.2650","4. close":"272.2400","5. volume":"5394121"},"2025-09-22":{"1. open":"266.6200","2. high":"272.3100","3. low":"266.0000","4. close":"271.3700","5. volume":"5030540"},"2025-09-19":{"1. open":"266.0500","2. high":"267.8700","3. low":"263.6400","4. close":"266.4000","5. volume":"9858112"},"2025-09-18":{"1. open":"258.8600","2. high":"265.2300","3. low":"256.8004","4. close":"265.0000","5. volume":"4988421"},"2025-09-17":{"1. open":"257.4950","2. high":"260.9644","3. low":"257.0100","4. close":"259.0800","5. volume":"3974785"},"2025-09-16":{"1. open":"256.2600","2. high":"258.0000","3. low":"254.4100","4. close":"257.5200","5. volume":"2719918"},"2025-09-15":{"1. open":"254.0200","2. high":"259.0500","3. low":"254.0000","4. close":"256.2400","5. volume":"4028365"},"2025-09-12":{"1. open":"256.9500","2. high":"257.2500","3. low":"252.4250","4. close":"253.4400","5. volume":"3433300"},"2025-09-11":{"1. open":"257.5600","2. high":"258.5450","3. low":"255.6550","4. close":"257.0100","5. volume":"3576048"},"2025-09-10":{"1. open":"259.6500","2. high":"260.0800","3. low":"254.5600","4. close":"256.8800","5. volume":"5185420"},"2025-09-09":{"1. open":"256.1200","2. high":"260.6600","3. low":"254.8800","4. close":"259.1100","5. volume":"4931105"},"2025-09-08":{"1. open":"248.6300","2. high":"257.1500","3. low":"247.0200","4. close":"256.0900","5. volume":"6940270"},"2025-09-05":{"1. open":"248.2300","2. high":"249.0300","3. low":"245.4500","4. close":"248.5300","5. volume":"3147478"},"2025-09-04":{"1. open":"245.4200","2. high":"249.2800","3. low":"242.8500","4. close":"247.1800","5. volume":"4765087"},"2025-09-03":{"1. open":"240.0200","2. high":"244.2500","3. low":"239.4100","4. close":"244.1000","5. volume":"3156289"},"2025-09-02":{"1. open":"240.9000","2. high":"241.5500","3. low":"238.2500","4. close":"241.5000","5. volume":"3469501"},"2025-08-29":{"1. open":"245.2300","2. high":"245.4599","3. low":"241.7200","4. close":"243.4900","5. volume":"2967558"},"2025-08-28":{"1. open":"245.4300","2. high":"245.8800","3. low":"243.3600","4. close":"245.7300","5. volume":"2820817"},"2025-08-27":{"1. open":"242.8700","2. high":"245.9600","3. low":"242.0000","4. close":"244.8400","5. volume":"3698372"},"2025-08-26":{"1. open":"241.0200","2. high":"244.9800","3. low":"240.3800","4. close":"242.6300","5. volume":"5386582"},"2025-08-25":{"1. open":"242.5650","2. high":"242.5650","3. low":"239.4300","4. close":"239.4300","5. volume":"3513327"},"2025-08-22":{"1. open":"240.7400","2. high":"243.6800","3. low":"240.2200","4. close":"242.0900","5. volume":"3134882"},"2025-08-21":{"1. open":"242.2100","2. high":"242.5000","3. low":"238.6500","4. close":"239.4000","5. volume":"2991902"},"2025-08-20":{"1. open":"242.1100","2. high":"242.8800","3. low":"240.3400","4. close":"242.5500","5. volume":"3240064"},"2025-08-19":{"1. open":"240.0000","2. high":"242.8300","3. low":"239.4900","4. close":"241.2800","5. volume":"3328305"},"2025-08-18":{"1. open":"239.5700","2. high":"241.4200","3. low":"239.1158","4. close":"239.4500","5. volume":"3569594"},"2025-08-15":{"1. open":"237.6100","2. high":"240.6200","3. low":"236.7700","4. close":"239.7200","5. volume":"4344322"},"2025-08-14":{"1. open":"238.2500","2. high":"239.0000","3. low":"235.6200","4. close":"237.1100","5. volume":"4556725"},"2025-08-13":{"1. open":"236.2000","2. high":"240.8411","3. low":"236.2000","4. close":"240.0700","5. volume":"5663562"},"2025-08-12":{"1. open":"236.5300","2. high":"237.9600","3. low":"233.3600","4. close":"234.7700","5. volume":"8800597"},"2025-08-11":{"1. open":"242.2400","2. high":"243.1500","3. low":"234.7000","4. close":"236.3000","5. volume":"9381960"},"2025-08-08":{"1. open":"248.8800","2. high":"249.4800","3. low":"241.6500","4. close":"242.2700","5. volume":"6828390"},"2025-08-07":{"1. open":"252.8100","2. high":"255.0000","3. low":"248.8750","4. close":"250.1600","5. volume":"6251285"},"2025-08-06":{"1. open":"251.5300","2. high":"254.3200","3. low":"249.2800","4. close":"252.2800","5. volume":"3692105"},"2025-08-05":{"1. open":"252.0000","2. high":"252.8000","3. low":"248.9950","4. close":"250.6700","5. volume":"5823016"},"2025-08-04":{"1. open":"251.0500","2. high":"252.0800","3. low":"248.1100","4. close":"251.9800","5. volume":"5280588"},"2025-08-01":{"1. open":"251.4050","2. high":"251.4791","3. low":"245.6100","4. close":"250.0500","5. volume":"9683404"},"2025-07-31":{"1. open":"259.5700","2. high":"259.9900","3. low":"252.2200","4. close":"253.1500","5. volume":"6739092"},"2025-07-30":{"1. open":"261.6000","2. high":"262.0000","3. low":"258.9000","4. close":"260.2600","5. volume":"3718290"},"2025-07-29":{"1. open":"264.3000","2. high":"265.7999","3. low":"261.0200","4. close":"262.4100","5. volume":"4627265"},"2025-07-28":{"1. open":"260.3000","2. high":"264.0000","3. low":"259.6100","4. close":"263.2100","5. volume":"5192516"},"2025-07-25":{"1. open":"260.0200","2. high":"260.8000","3. low":"256.3500","4. close":"259.7200","5. volume":"7758653"},"2025-07-24":{"1. open":"261.2500","2. high":"262.0486","3. low":"252.7500","4. close":"260.5100","5. volume":"22647720"},"2025-07-23":{"1. open":"284.3000","2. high":"288.0800","3. low":"281.4400","4. close":"282.0100","5. volume":"8105906"},"2025-07-22":{"1. open":"284.7400","2. high":"284.8800","3. low":"281.2500","4. close":"281.9600","5. volume":"4824219"},"2025-07-21":{"1. open":"286.2900","2. high":"287.7300","3. low":"284.3800","4. close":"284.7100","5. volume":"3051791"},"2025-07-18":{"1. open":"283.3800","2. high":"287.1600","3. low":"282.2200","4. close":"285.8700","5. volume":"4478165"},"2025-07-17":{"1. open":"281.5000","2. high":"283.4566","3. low":"280.9000","4. close":"282.0000","5. volume":"3337168"},"2025-07-16":{"1. open":"282.7500","2. high":"283.8700","3. low":"279.8700","4. close":"281.9200","5. volume":"2804831"},"2025-07-15":{"1. open":"283.7700","2. high":"284.1550","3. low":"280.7301","4. close":"282.7000","5. volume":"2864106"},"2025-07-14":{"1. open":"282.8300","2. high":"284.9250","3. low":"281.7100","4. close":"283.7900","5. volume":"2857401"},"2025-07-11":{"1. open":"285.0100","2. high":"287.4300","3. low":"282.9200","4. close":"283.5900","5. volume":"3790679"},"2025-07-10":{"1. open":"288.9000","2. high":"288.9000","3. low":"282.2100","4. close":"287.4300","5. volume":"3489068"},"2025-07-09":{"1. open":"291.3900","2. high":"291.6000","3. low":"288.6300","4. close":"290.1400","5. volume":"2971309"},"2025-07-08":{"1. open":"293.1000","2. high":"295.6100","3. low":"289.4900","4. close":"290.4200","5. volume":"2925329"},"2025-07-07":{"1. open":"292.5000","2. high":"295.2199","3. low":"290.3607","4. close":"292.4700","5. volume":"4488064"},"2025-07-03":{"1. open":"287.9400","2. high":"292.3200","3. low":"287.9000","4. close":"291.9700","5. volume":"1853289"},"2025-07-02":{"1. open":"290.0000","2. high":"290.1900","3. low":"286.9000","4. close":"287.6500","5. volume":"3257515"},"2025-07-01":{"1. open":"294.5500","2. high":"295.1081","3. low":"290.0800","4. close":"291.2000","5. volume":"3272797"},"2025-06-30":{"1. open":"290.9300","2. high":"294.8100","3. low":"290.0000","4. close":"294.7800","5. volume":"3495386"},"2025-06-27":{"1. open":"292.9700","2. high":"293.1200","3. low":"288.5200","4. close":"289.7000","5. volume":"3562501"},"2025-06-26":{"1. open":"291.8000","2. high":"292.9100","3. low":"290.1650","4. close":"291.9300","5. volume":"3621110"},"2025-06-25":{"1. open":"294.4900","2. high":"296.1600","3. low":"289.5000","4. close":"291.0600","5. volume":"3862309"},"2025-06-24":{"1. open":"290.4600","2. high":"294.3399","3. low":"288.4100","4. close":"293.7900","5. volume":"4219120"},"2025-06-23":{"1. open":"281.6500","2. high":"289.5800","3. low":"280.2100","4. close":"289.1800","5. volume":"3786159"},"2025-06-20":{"1. open":"279.2800","2. high":"284.1200","3. low":"277.2000","4. close":"280.9700","5. volume":"7676962"},"2025-06-18":{"1. open":"285.0000","2. high":"286.9100","3. low":"282.9400","4. close":"283.2100","5. volume":"3534110"},"2025-06-17":{"1. open":"281.1500","2. high":"284.7899","3. low":"281.0001","4. close":"283.0500","5. volume":"3069556"},"2025-06-16":{"1. open":"279.3050","2. high":"284.5000","3. low":"278.6657","4. close":"281.8300","5. volume":"3685321"},"2025-06-13":{"1. open":"278.2050","2. high":"279.8400","3. low":"275.8300","4. close":"277.2200","5. volume":"3243824"},"2025-06-12":{"1. open":"281.5300","2. high":"283.0600","3. low":"279.8300","4. close":"281.0300","5. volume":"3418007"},"2025-06-11":{"1. open":"276.7000","2. high":"281.7500","3. low":"275.1100","4. close":"281.5200","5. volume":"4656034"},"2025-06-10":{"1. open":"273.1900","2. high":"277.4700","3. low":"272.5600","4. close":"276.2400","5. volume":"5163507"},"2025-06-09":{"1. open":"268.1000","2. high":"273.4700","3. low":"266.7100","4. close":"272.0800","5. volume":"4331464"},"2025-06-06":{"1. open":"267.9900","2. high":"270.1700","3. low":"267.5300","4. close":"268.8700","5. volume":"2495543"},"2025-06-05":{"1. open":"265.2000","2. high":"267.5100","3. low":"265.1000","4. close":"266.8600","5. volume":"2659478"},"2025-06-04":{"1. open":"264.9000","2. high":"267.0000","3. low":"264.7900","4. close":"265.5200","5. volume":"2588741"},"2025-06-03":{"1. open":"263.3500","2. high":"265.5600","3. low":"262.5800","4. close":"265.2000","5. volume":"2494922"},"2025-06-02":{"1. open":"257.8500","2. high":"263.9760","3. low":"257.2200","4. close":"263.9000","5. volume":"2831881"},"2025-05-30":{"1. open":"258.7500","2. high":"260.1200","3. low":"257.1000","4. close":"259.0600","5. volume":"9668923"},"2025-05-29":{"1. open":"260.7500","2. high":"261.1300","3. low":"256.7700","4. close":"258.6900","5. volume":"2295228"},"2025-05-28":{"1. open":"263.1600","2. high":"265.0000","3. low":"259.9400","4. close":"260.2400","5. volume":"2318437"},"2025-05-27":{"1. open":"261.0000","2. high":"263.7869","3. low":"259.6300","4. close":"263.2300","5. volume":"3284216"},"2025-05-23":{"1. open":"258.5800","2. high":"259.8696","3. low":"255.7900","4. close":"258.6300","5. volume":"2722721"},"2025-05-22":{"1. open":"260.7700","2. high":"261.2711","3. low":"257.9100","4. close":"258.3700","5. volume":"3091253"},"2025-05-21":{"1. open":"264.9700","2. high":"265.6499","3. low":"260.4100","4. close":"260.8700","5. volume":"3753904"},"2025-05-20":{"1. open":"267.4000","2. high":"269.2800","3. low":"265.6201","4. close":"266.9500","5. volume":"2437860"},"2025-05-19":{"1. open":"265.4500","2. high":"269.1350","3. low":"265.0800","4. close":"268.4100","5. volume":"3198903"},"2025-05-16":{"1. open":"266.3500","2. high":"267.9800","3. low":"264.5900","4. close":"266.7600","5. volume":"3817937"},"2025-05-15":{"1. open":"259.0100","2. high":"267.4300","3. low":"258.6100","4. close":"266.6800","5. volume":"4856276"},"2025-05-14":{"1. open":"257.6000","2. high":"260.5500","3. low":"256.2200","4. close":"257.8200","5. volume":"3635124"},"2025-05-13":{"1. open":"254.4300","2. high":"259.5800","3. low":"252.8800","4. close":"258.5900","5. volume":"3521389"}}}

sma_data = {"Meta Data":{"1: Symbol":"IBM","2: Indicator":"Simple Moving Average (SMA)","3: Last Refreshed":"2025-10-03","4: Interval":"weekly","5: Time Period":10,"6: Series Type":"open","7: Time Zone":"US/Eastern"},"Technical Analysis: SMA":{"2025-10-03":{"SMA":"252.8374"},"2025-09-26":{"SMA":"252.6692"},"2025-09-19":{"SMA":"254.0954"},"2025-09-12":{"SMA":"257.7420"},"2025-09-05":{"SMA":"261.7716"},"2025-08-29":{"SMA":"265.6527"},"2025-08-22":{"SMA":"269.1343"},"2025-08-15":{"SMA":"271.8027"},"2025-08-08":{"SMA":"273.1861"},"2025-08-01":{"SMA":"274.1743"},"2025-07-25":{"SMA":"274.6857"},"2025-07-18":{"SMA":"271.3300"},"2025-07-11":{"SMA":"267.2858"},"2025-07-03":{"SMA":"261.2080"},"2025-06-27":{"SMA":"255.7996"},"2025-06-20":{"SMA":"251.4810"},"2025-06-13":{"SMA":"245.3701"},"2025-06-06":{"SMA":"242.6901"},"2025-05-30":{"SMA":"241.4789"},"2025-05-23":{"SMA":"240.1462"}}}


ema_data = {"Meta Data":{"1: Symbol":"IBM","2: Indicator":"Exponential Moving Average (EMA)","3: Last Refreshed":"2025-10-03","4: Interval":"weekly","5: Time Period":10,"6: Series Type":"open","7: Time Zone":"US/Eastern"},"Technical Analysis: EMA":{"2025-10-03":{"EMA":"260.7207"},"2025-09-26":{"EMA":"255.1031"},"2025-09-19":{"EMA":"252.5437"},"2025-09-12":{"EMA":"252.2157"},"2025-09-05":{"EMA":"253.0125"},"2025-08-29":{"EMA":"255.7041"},"2025-08-22":{"EMA":"258.6240"},"2025-08-15":{"EMA":"262.8582"},"2025-08-08":{"EMA":"267.4400"},"2025-08-01":{"EMA":"271.4664"},"2025-07-25":{"EMA":"274.3462"},"2025-07-18":{"EMA":"272.1301"},"2025-07-11":{"EMA":"270.1852"},"2025-07-03":{"EMA":"265.6740"},"2025-06-27":{"EMA":"260.5068"},"2025-06-20":{"EMA":"256.2393"},"2025-06-13":{"EMA":"251.5411"},"2025-06-06":{"EMA":"248.2716"},"2025-05-30":{"EMA":"246.5377"},"2025-05-23":{"EMA":"243.7233"}}}


rsi_data = {"Meta Data":{"1: Symbol":"IBM","2: Indicator":"Relative Strength Index (RSI)","3: Last Refreshed":"2025-10-03","4: Interval":"weekly","5: Time Period":10,"6: Series Type":"open","7: Time Zone":"US/Eastern Time"},"Technical Analysis: RSI":{"2025-10-03":{"RSI":"67.6313"},"2025-09-26":{"RSI":"58.4411"},"2025-09-19":{"RSI":"50.1612"},"2025-09-12":{"RSI":"46.0208"},"2025-09-05":{"RSI":"39.5374"},"2025-08-29":{"RSI":"40.4799"},"2025-08-22":{"RSI":"38.0907"},"2025-08-15":{"RSI":"39.3583"},"2025-08-08":{"RSI":"42.7545"},"2025-08-01":{"RSI":"47.5445"},"2025-07-25":{"RSI":"66.3390"},"2025-07-18":{"RSI":"64.6654"},"2025-07-11":{"RSI":"73.9081"},"2025-07-03":{"RSI":"73.3515"},"2025-06-27":{"RSI":"69.9405"},"2025-06-20":{"RSI":"69.0393"},"2025-06-13":{"RSI":"64.4562"},"2025-06-06":{"RSI":"59.5233"},"2025-05-30":{"RSI":"61.8994"},"2025-05-23":{"RSI":"65.2089"}}}
